In [26]:
import os
import pandas as pd

In [95]:
# Step 1. Create a `shift` table for normalizing the scores based on Scale Score Range
idx = pd.MultiIndex.from_product([[3,4,5,6,7,8],['ELA','Math']], names=['Grade', 'Subject'])
shift = pd.DataFrame([2114,2189,2131,2204,2201,2219,2210,2235,2258,2250,2288,2265], idx, ['Point'])
shift[:3]

Point
Grade Subject       
3     ELA       2114
      Math      2189
4     ELA       2131

In [27]:
# Step 2. Read "Education Directory" from following CT Open Data Portal
ed = pd.read_csv('Education_Directory.csv')
ed['Geo'] = ed['Location'].str.split('\n',expand=True)[2] # extract the Geo info from Location
ed[['lat','lon']] = ed['Geo'].str[1:-1].str.split(',', expand=True) # extract the lat & lon from Geo
ed[:3]

,District Name,School Name,Organization Type,Organization Code,Address,Town,Zipcode,Phone,PreKindergarten,Kindergarten,...,Grade 9,Grade 10,Grade 11,Grade 12,Student Open Date,Interdistrict Magnet,Location,Geo,lat,lon
0,Stamford School District,Turn of River School,Public Schools,1355511,117 Vine Rd.,Stamford,06905,203-977-4284,0,0,...,0,0,0,0,1984-07-01T00:00:00.000,0.0,"117 Vine Rd.\nStamford, CT 06905\n(41.100654, ...","(41.100654, -73.543187)",41.100654,-73.543187
1,Voluntown School District,Voluntown School District,Public School Districts,1470011,"Po Box 129, 195 Main St.",Voluntown,06384,860-376-9167,1,1,...,0,0,0,0,1996-07-01T00:00:00.000,0.0,NaN,NaN,NaN,NaN
2,Capitol Region Education Council,Academy of Aerospace and Engineering,Regional Education Service Center Schools,2415114,1101 Kennedy Road,Windsor,06095,860-243-0857,0,0,...,1,1,1,1,2008-08-25T00:00:00.000,1.0,"1101 Kennedy Road\nWindsor, CT 06095\n(41.8959...","(41.895931, -72.65199)",41.895931,-72.65199


In [5]:
# Step 3. Extract town income data from wikipedia   
town_income=pd.read_html('https://en.wikipedia.org/wiki/List_of_Connecticut_locations_by_per_capita_income')[2]
town_income[:3]

,Rank,Town,Unnamed: 2,County,Per capita income,Median household income,Median family income,Population,Number of households
0,1.0,New Canaan,Town,Fairfield,"$105,846","$174,611","$211,875",19738,6857
1,2.0,Darien,Town,Fairfield,"$105,846","$208,848","$343,456",20732,6555
2,3.0,Greenwich,Town,Fairfield,"$90,087","$128,153","$167,825",61171,22083


In [93]:
# Step 4. Read the manually formatted smarterbalanced.csv (see README's `Manual Data Cleaning`)
# Then we replace "*" with "" to indicate NULL and save the cleaned file as smarterbalanced_cleaned.csv
fin = open("smarterbalanced.csv", "rt")
fout = open("smarterbalanced_cleaned.csv", "wt")
for line in fin:
	fout.write(line.replace('*',''))
fin.close()
fout.close()

In [96]:
# Step 5. Data Cleaning 
# removing mostly blank groups: Indians and multi-racial
df = pd.read_csv('smarterbalanced_cleaned.csv', index_col=[0,1,2,3],header=[0,1])
df = df.loc[(slice(None), slice(None) , ['Asian','Black or African American','Hispanic/Latino of any race','White'])]
student_no = df.loc[(),('2021','Total Number with Scored Tests')]

# only focus on average VSS, the raw score
df = df.loc[(),(slice(None),'Average VSS')]
df['StudentNo'] = student_no
df.columns = df.columns.get_level_values(0) # `flatten` the columns

# remove the rows with all NaN
df = df[~(df['2015'].isna() & df['2016'].isna() & df['2017'].isna() & df['2018'].isna() & df['2021'].isna())]

In [99]:
df

,Grade,Subject,Race/Ethnicity,District,2015,2016,2017,2018,2021,StudentNo
0,6,ELA,Asian,Area Cooperative Educational Services,339.0,NaN,NaN,NaN,NaN,NaN
1,6,Math,Asian,Area Cooperative Educational Services,298.0,NaN,NaN,NaN,NaN,NaN
2,8,ELA,Asian,Area Cooperative Educational Services,355.0,NaN,NaN,NaN,NaN,NaN
3,8,Math,Asian,Area Cooperative Educational Services,359.0,NaN,NaN,NaN,NaN,NaN
4,3,ELA,Asian,Avon School District,384.0,397.0,402.0,376.0,360.0,40.0
...,...,...,...,...,...,...,...,...,...,...
3977,6,Math,White,Woodstock School District,283.0,299.0,297.0,311.0,287.0,73.0
3978,7,ELA,White,Woodstock School District,336.0,310.0,324.0,322.0,314.0,75.0
3979,7,Math,White,Woodstock School District,338.0,303.0,325.0,296.0,288.0,75.0
3980,8,ELA,White,Woodstock School District,317.0,290.0,297.0,303.0,305.0,99.0


In [98]:
# Step 6. Normalize the scores 
# by appending the Point column from 'shift'
df.loc[:, 'Point'] = shift.Point 

# and then subtracting the Point column from the scores
df['2015'] = df['2015'] - df['Point']
df['2016'] = df['2016'] - df['Point']
df['2017'] = df['2017'] - df['Point']
df['2018'] = df['2018'] - df['Point']
df['2021'] = df['2021'] - df['Point']
df.drop('Point',axis=1, inplace=True) # Remove the shift

df.reset_index(inplace=True) # reset the index, to be able to join

In [76]:
# what's wrong with the data?
pd.merge(df,ed[['District Name','Town','Zipcode','lat','lon']], how='left', left_on='District', right_on='District Name').shape

(36158, 14)

In [77]:
# Step 7. Data Merge
# Merge with the Education Directory to get the town, zipcode, lat & lon
ed_per_district = ed.groupby(['District Name'])[['Town','Zipcode','lat','lon']].first().reset_index().rename(columns={"District Name": "District"})
df = pd.merge(df,ed_per_district[['District','Town','Zipcode','lat','lon']], how='left', left_on='District', right_on='District')
# Merges with the Education Directory & town income data
df = pd.merge(df,town_income[['Town','Per capita income','Population']], how='left', left_on='Town', right_on='Town')
# change the currency to int
df['Per capita income'] = df[~df['Per capita income'].isna()]['Per capita income'].apply(
    lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(int)


In [ ]:
# Step 8. Save the final file, and remove the intermediate file
df.to_csv("sb_final.csv",index=False)
os.remove("smarterbalanced_cleaned.csv")

## Below are testing code

In [78]:
print(df.shape)
df[:4]

(4032, 15)


,Grade,Subject,Race/Ethnicity,District,2015,2016,2017,2018,2021,Town,Zipcode,lat,lon,Per capita income,Population
0,6,ELA,Asian,Area Cooperative Educational Services,339.0,NaN,NaN,NaN,NaN,North Haven,06473,41.397451,-72.838237,38286.0,24093.0
1,6,Math,Asian,Area Cooperative Educational Services,298.0,NaN,NaN,NaN,NaN,North Haven,06473,41.397451,-72.838237,38286.0,24093.0
2,8,ELA,Asian,Area Cooperative Educational Services,355.0,NaN,NaN,NaN,NaN,North Haven,06473,41.397451,-72.838237,38286.0,24093.0
3,8,Math,Asian,Area Cooperative Educational Services,359.0,NaN,NaN,NaN,NaN,North Haven,06473,41.397451,-72.838237,38286.0,24093.0
